# Import TOL Performance by Area

In [7]:
import cx_Oracle
import pandas as pd
import numpy as np
import configparser

## Parameter

In [8]:
config = configparser.ConfigParser()
config.read('../my_config.ini')
config.sections()

username = config['TDMDBPR']['username']
password = config['TDMDBPR']['password']
db = config['TDMDBPR']['db']
host = config['TDMDBPR']['host']
port = config['TDMDBPR']['port']
as_week = config['PARAM']['as_week']

In [9]:
raw_dir = f"../data/raw"
raw_file_name = f'Raw-TOL Perf Area_Wk {as_week}'

## Query

In [10]:
conStr = f'{username}/{password}@{host}:{port}/{db}'
conn = cx_Oracle.connect(conStr)
print(f'{db} : Connected')
cur = conn.cursor()

v_week = (as_week, )


try:
    print(f'\nExecute query as Week {v_week[0]}...')

    sqlTxt = """
        SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, DAY_DESC, MONTH_DESC, QUARTER_DESC, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME, ACTUAL_AS_OF
            , AREA_NO, AREA_TYPE, AREA_CD, AREA_NAME
            , AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD 
            , ACTUAL_SNAP, ACTUAL_AGG, TARGET_SNAP, TARGET_AGG, PPN_TM
        FROM ( 
            SELECT TM_KEY_DAY, TM_KEY_WK, TM_KEY_MTH, TM_KEY_QTR, TM_KEY_YR, DAY_DESC, MONTH_DESC, QUARTER_DESC, CENTER, METRIC_GRP, COMP_CD, PRODUCT_GRP, METRIC_CD, METRIC_NAME, ACTUAL_AS_OF 
                , CASE WHEN AREA_TYPE = 'P' THEN 1 WHEN AREA_TYPE = 'G' THEN 2 WHEN AREA_TYPE = 'H' THEN 3 ELSE 0 END AREA_NO
                , AREA_TYPE, AREA_CD, AREA_NAME, METRIC_NAME_GROUP, PPN_TM, AGG_TYPE, RR_IND, GRY_IND, UOM, PERIOD, ACTUAL_SNAP, ACTUAL_AGG, TARGET_SNAP, TARGET_AGG
            FROM GEOSPCAPPO.AGG_PERFORMANCE_NEWCO NOLOCK 
            WHERE PRODUCT_GRP = 'TOL' --All about 500,000 rows, Analysis about 120,000 rows
            AND (METRIC_CD IN ('TB3R000100', 'TB3R000101', 'TB3R000102') --"TOL Revenue". "TOL New Revenue", "TOL Exisitng Revenue"
                OR METRIC_CD IN ('TB3S000100', 'TB3S000101', 'TB3S000400', 'TB3S000500') --"TOL Connected Subs", "TOL Registration Subs", "TOL Net Add", "TOL Active Subs"
                OR METRIC_CD IN ('TSER13100', 'TB3R000600')) --"TOL Churn Subs", "TOL Connected 1MTH RC"
            AND AREA_TYPE IN ('P', 'G', 'H')
        ) TOL_ANALYSIS 
    """
    
    cur.execute(sqlTxt)

    rows = cur.fetchall()

    print(f'\nDone !!!')
    
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])


except cx_Oracle.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{db} : Disconnected')

TDMDBPR : Connected

Execute query as Week 2023027...

Done !!!

TDMDBPR : Disconnected


In [11]:
df.shape
# df.columns
# df.dtypes
# df.describe()

# df.head()

(139896, 29)

## Generate Output file

In [12]:
op_raw_file = f'{raw_dir}/{raw_file_name}.csv'

df.to_csv(f'{op_raw_file}', index=False, encoding='utf-8')

print(f'\n"{op_raw_file}" is generated')


"../data/raw/Raw-TOL Perf Area_Wk 2023027.csv" is generated
